In [1]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import torch

In [2]:
df = pd.read_csv("../Data for analysis/data_for_further_analysis.csv")
df["text"] = df["title"].fillna('') + ". " + df["clean_text"].fillna('')
texts = df["text"].tolist()


In [3]:
device = 0 if torch.cuda.is_available() else -1
classifier = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=device,
    truncation=True
)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
batch_size = 32
all_sentiments = []

for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i+batch_size]
    outputs = classifier(batch)
    all_sentiments.extend(outputs)

100%|██████████| 2977/2977 [48:50<00:00,  1.02it/s]  


In [6]:
labels = [out["label"] for out in all_sentiments]
scores = [out["score"] for out in all_sentiments]

df_sentiment = pd.DataFrame({
    "post_text": texts,
    "sentiment_label": labels,
    "sentiment_score": scores
})

from pathlib import Path
output_path = Path("../DistillBert/Output/distilbert_sentiment_scores.csv")
df_sentiment.to_csv(output_path, index=False)
print(f"Saved → {output_path}")


Saved → ../DistillBert/Output/distilbert_sentiment_scores.csv
